Interesting lecture: https://machinelearningmastery.com/k-fold-cross-validation/

In [1]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
import os
import math
from sklearn import preprocessing, utils
import warnings
from DataGenerator import ImgListDataGen
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

Using TensorFlow backend.


### First, it is important to understand differences between ShuffleSplit and KFold

In [2]:
splits = 5

tx = range(10)
ty = [0] * 5 + [1] * 5

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import datasets

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
shufflesplit = StratifiedShuffleSplit(n_splits=splits, random_state=42, test_size=2)

print("KFold")
for train_index, test_index in kfold.split(tx, ty):
    print("TRAIN:", train_index, "TEST:", test_index)

print("Shuffle Split")
for train_index, test_index in shufflesplit.split(tx, ty):
    print("TRAIN:", train_index, "TEST:", test_index)

KFold
TRAIN: [0 2 3 4 5 7 8 9] TEST: [1 6]
TRAIN: [0 1 2 3 5 6 7 8] TEST: [4 9]
TRAIN: [0 1 3 4 5 6 8 9] TEST: [2 7]
TRAIN: [1 2 3 4 6 7 8 9] TEST: [0 5]
TRAIN: [0 1 2 4 5 6 7 9] TEST: [3 8]
Shuffle Split
TRAIN: [8 4 1 0 6 5 7 2] TEST: [3 9]
TRAIN: [7 0 3 9 4 5 1 6] TEST: [8 2]
TRAIN: [1 2 5 6 4 8 9 0] TEST: [3 7]
TRAIN: [4 6 7 8 3 5 1 2] TEST: [9 0]
TRAIN: [7 2 6 5 4 3 0 9] TEST: [1 8]


In [3]:
# In KFolds, each test set should not overlap, even with shuffle. 
# With KFolds and shuffle, the data is shuffled once at the start, and then divided into the number of desired splits. 
# The test data is always one of the splits, the train data is the rest.

# In ShuffleSplit, the data is shuffled every time, and then split. 
# This means the test sets may overlap between the splits:
#                                Test, first row 3 and third row 3, first row 9 and fourth row 9.

In [4]:
# Thus, in ShuffleSplit test_size can be specified, for instance .2 means 1-.2 = .8 for training.
# While, KFolds depends on the size of the data and K (test set should not overlap), thus, if data is len 10, 
# and K is 5, then test size is 10/5=2 in order to never overlap.

### Second, let's create the K-Fold cross validation

In [5]:
DB_Path = '/Users/dfreire/Dropbox/Datasets/small_dataset/train'

In [6]:
def Load_KFold(DB_path, k=2):
    #Numeric labels
    lab = preprocessing.LabelEncoder()
    lab.fit(os.listdir(DB_path))
    labels = lab.transform(os.listdir(DB_path))
    inv = lab.inverse_transform(labels)
    class_dict = dict(zip(inv, labels))
    print(class_dict)
    #inv_class_dict = {k: v for v,k in class_dict.items()} to decode labels if necessary

    #Read each folder
    data=[]
    labels=[]
    for class_ in os.listdir(DB_path):
        dat = [os.path.join(DB_path, class_, img) for img in os.listdir(os.path.join(DB_path, class_))]
        lab = np.ones(len(dat))*class_dict[class_]
        
        labels = np.concatenate((labels,lab))
        data = data + dat
    
    print(len(data))
    print(len(labels))
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1234).split(data, labels))
    
    return folds, np.array(data), labels.astype(int)

In [7]:
k=4
folds, data, labels = Load_KFold(DB_path=DB_Path, k=k)

{'cats': 0, 'dogs': 1}
2000
2000


In [8]:
print('There are {} Folds'.format(len(folds)))
print('Train data contains {} samples'.format(len(folds[0][0])))
print('Test data contains {} samples'.format(len(folds[0][1])))
print('Test data samples are aprox obtained from {}'.format(math.ceil(len(data)/k)))

There are 4 Folds
Train data contains 1500 samples
Test data contains 500 samples
Test data samples are aprox obtained from 500


In [9]:
#Let's check the test set of the first fold
unique, counts = np.unique(labels[folds[0][1]], return_counts=True)
dict(zip(unique, counts))
#250 items of class 0 and 250 of class 1

{0: 250, 1: 250}

In [10]:
for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    
    X_train_cv = data[train_idx]
    y_train_cv = labels[train_idx]
    X_valid_cv = data[val_idx]
    y_valid_cv= labels[val_idx]
    
    X_train_cv, y_train_cv = utils.shuffle(X_train_cv, y_train_cv, random_state=1234)

    unique, counts = np.unique(y_train_cv, return_counts=True)
    print('For training, {0} samples: {1}'.format(len(X_train_cv), dict(zip(unique, counts))))
    
    
    unique, counts = np.unique(y_valid_cv, return_counts=True)
    print('For validation, {0} samples: {1}'.format(len(X_valid_cv), dict(zip(unique, counts))))
    
    
    print(X_train_cv[:5])


Fold  0
For training, 1500 samples: {0: 750, 1: 750}
For validation, 500 samples: {0: 250, 1: 250}
['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.189.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.553.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.53.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.102.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.774.jpg']

Fold  1
For training, 1500 samples: {0: 750, 1: 750}
For validation, 500 samples: {0: 250, 1: 250}
['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.198.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.539.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.52.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.102.jpg'
 '/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.781.jpg']

Fold  2
For training, 1500 samples: {0: 750, 1: 750}
For validation, 500 sample

The general procedure is as follows:

Shuffle the dataset randomly. ok
Split the dataset into k groups ok
For each unique group:
Take the group as a hold out or test data set
Take the remaining groups as a training data set
Fit a model on the training set and evaluate it on the test set
Retain the evaluation score and discard the model
Summarize the skill of the model using the sample of model evaluation scores

In [11]:
from keras import Input, optimizers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.models import Model

In [18]:
def get_model():
    
    entrada= Input(shape=(150,150,3))
    
    conv = Conv2D(filters=32, kernel_size=3, activation='relu', name='conv_1')(entrada)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_1')(conv)
    
    conv = Conv2D(filters=64, kernel_size=3, activation='relu', name='conv_2')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_2')(conv)   
    
    conv = Conv2D(filters=128, kernel_size=3, activation='relu', name='conv_3')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_3')(conv)
        
    conv = Conv2D(filters=128, kernel_size=3, activation='relu', name='conv_4')(maxpool)
    maxpool = MaxPool2D(pool_size=2, strides=2, name='maxpool_4')(conv)
    
    flat = Flatten(name='flatten')(maxpool)
    #drop = Dropout(rate=.5, name='dropout')(flat)
    
    dense = Dense(units=512, activation='relu', name='Dense1')(flat)#(drop)
    output = Dense(units=1, activation='sigmoid', name='output')(dense)
    #output = Dense(units=2, activation='softmax', name='output')(dense)
    
    model = Model(entrada, output)
    
    #model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])
    model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [19]:
batch_size=20
# Parameters
train_params = {
    'rescale': 1./255,
    'batch_size': batch_size,
    'n_classes': 2, 
    'target_size': (150,150,3),
    'aug_mode': None,
    'class_mode': 'binary', #'categorical',
    'shuffle': False}
val_params = {
    'rescale': 1./255,
    'batch_size': batch_size,
    'n_classes': 2, 
    'target_size': (150,150,3),
    'aug_mode': None,
    'class_mode': 'binary', #'categorical',
    'shuffle': False}

In [75]:
batch_size=20
# Parameters
train_params = {
    'rescale': 1./255,
    'batch_size': batch_size,
    'target_size': (150,150),
    'aug_mode': None,
    'class_mode': 'binary', #'categorical',
    'shuffle': False}
val_params = {
    'rescale': 1./255,
    'batch_size': batch_size,
    'target_size': (150,150),
    'aug_mode': None,
    'class_mode': 'binary', #'categorical',
    'shuffle': False}

In [78]:
for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    X_train_cv = data[train_idx]
    y_train_cv = labels[train_idx]
    X_valid_cv = data[val_idx]
    y_valid_cv= labels[val_idx]
    
    X_train_cv, y_train_cv = utils.shuffle(X_train_cv, y_train_cv, random_state=12)
    
    #name_weights = "final_model_fold" + str(j) + "_weights.h5"
    #callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
    training_gen = ImgListDataGenFrf(img_files = X_train_cv, labels=y_train_cv, **train_params)
    validation_gen = ImgListDataGenFrf(img_files = X_valid_cv, labels=y_valid_cv, **val_params) 
    print('Training')                                    
    model = get_model()
    model.fit_generator(
                training_gen,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=10,
                shuffle=True,
                verbose=1,
                validation_steps=10,
                validation_data = validation_gen)#,
                #callbacks = callbacks)
    
    print(model.evaluate(X_valid_cv, y_valid_cv))


Fold  0
Training
Epoch 1/10(150, 150, 3)

(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
(150, 150, 3)
18/75 [======>.......................] - ETA: 1:16 - loss: 7.2350 - acc: 0.5028(150, 150, 3)


KeyboardInterrupt: 

In [ ]:
#Inverse from to_categorical

In [24]:
from numpy.random import randint
from numpy import argmax
from keras.utils.np_utils import to_categorical
k = 8
n = 10
x = randint(0, k, (n,))
print(x)
cat = to_categorical(x, k)
print(cat)
inv = np.array([val for val in [argmax(i) for i in cat]]) ###
print(inv)

[2 1 4 4 2 5 3 1 2 7]
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
[2 1 4 4 2 5 3 1 2 7]


In [15]:
#Small generator example
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import keras


model = get_model()

# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

class CIFAR10Sequence(keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        print(idx)
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        k=np.array([resize(imread(file_name), (150, 150, 3)) for file_name in batch_x])
        #print(k.shape)
        #print(batch_y)

        return np.array([resize(imread(file_name), (150, 150, 3)) for file_name in batch_x]), np.array(batch_y)
    
    
    
Xtrain_gen = CIFAR10Sequence(data,labels,batch_size=10) # you can choose your batch size.
#Xvalidation_gen = detracSequence(X_validation,y_validation,batch_size=512)

model.fit_generator(generator=Xtrain_gen, epochs=10)

Epoch 1/10
183


//anaconda/envs/SCI_env/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
//anaconda/envs/SCI_env/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


41
9
165
102
150
15
  8/200 [>.............................] - ETA: 2:32 - loss: 8.0542 - acc: 0.500046


KeyboardInterrupt: 

In [50]:
class Iterator(object):
    """Base class for image data iterators.
    Every `Iterator` must implement the `_get_batches_of_transformed_samples`
    method.
    # Arguments
        n: Integer, total number of samples in the dataset to loop over.
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seeding for data shuffling.
    """

    def __init__(self, n, batch_size, shuffle, seed):
        self.n = n
        self.batch_size = batch_size
        self.seed = seed
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.index_array = None
        self.index_generator = self._flow_index()

    def _set_index_array(self):
        self.index_array = np.arange(self.n)
        if self.shuffle:
            self.index_array = np.random.permutation(self.n)

    def __getitem__(self, idx):
        if idx >= len(self):
            raise ValueError('Asked to retrieve element {idx}, '
                             'but the Sequence '
                             'has length {length}'.format(idx=idx,
                                                          length=len(self)))
        if self.seed is not None:
            np.random.seed(self.seed + self.total_batches_seen)
        self.total_batches_seen += 1
        if self.index_array is None:
            self._set_index_array()
        index_array = self.index_array[self.batch_size * idx:
                                       self.batch_size * (idx + 1)]
        return self._get_batches_of_transformed_samples(index_array)

    def __len__(self):
        return (self.n + self.batch_size - 1) // self.batch_size  # round up

    def on_epoch_end(self):
        self._set_index_array()

    def reset(self):
        self.batch_index = 0

    def _flow_index(self):
        # Ensure self.batch_index is 0.
        self.reset()
        while 1:
            if self.seed is not None:
                np.random.seed(self.seed + self.total_batches_seen)
            if self.batch_index == 0:
                self._set_index_array()

            current_index = (self.batch_index * self.batch_size) % self.n
            if self.n > current_index + self.batch_size:
                self.batch_index += 1
            else:
                self.batch_index = 0
            self.total_batches_seen += 1
            yield self.index_array[current_index:
                                   current_index + self.batch_size]

    def __iter__(self):
        # Needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)

    def _get_batches_of_transformed_samples(self, index_array):
        """Gets a batch of transformed samples.
        # Arguments
            index_array: Array of sample indices to include in batch.
        # Returns
            A batch of transformed samples.
        """
        raise NotImplementedError

In [77]:
import keras 
try:
    from PIL import ImageEnhance
    from PIL import Image as pil_image
except ImportError:
    pil_image = None
    ImageEnhance = None

if pil_image is not None:
    _PIL_INTERPOLATION_METHODS = {
        'nearest': pil_image.NEAREST,
        'bilinear': pil_image.BILINEAR,
        'bicubic': pil_image.BICUBIC,
    }
    # These methods were only introduced in version 3.4.0 (2016).
    if hasattr(pil_image, 'HAMMING'):
        _PIL_INTERPOLATION_METHODS['hamming'] = pil_image.HAMMING
    if hasattr(pil_image, 'BOX'):
        _PIL_INTERPOLATION_METHODS['box'] = pil_image.BOX
    # This method is new in version 1.1.3 (2013).
    if hasattr(pil_image, 'LANCZOS'):
        _PIL_INTERPOLATION_METHODS['lanczos'] = pil_image.LANCZOS

        
class ImgListDataGenFrf(Iterator):
    
    # Initialization
    def __init__(self, img_files, labels, rescale=1., aug_mode=None, 
                 target_size=(256, 256), color_mode='rgb',
                 classes=None, class_mode='categorical',
                 batch_size=32, shuffle=True, seed=None,
                 data_format='channels_last',
                 save_to_dir=None, save_prefix='', save_format='png',
                 follow_links=False,
                 interpolation='nearest',
                 dtype='float32'):
        
        self.img_files = img_files
        self.labels = labels
        self.rescale = rescale
        self.augmentation = aug_mode
        self.target_size = tuple(target_size)
        if color_mode not in {'rgb', 'rgba', 'grayscale'}:
            raise ValueError('Invalid color mode:', color_mode,
                             '; expected "rgb", "rgba", or "grayscale".')
        self.color_mode = color_mode
        self.data_format = data_format
        if self.color_mode == 'rgba':
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (4,)
            else:
                self.image_shape = (4,) + self.target_size
        elif self.color_mode == 'rgb':
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (3,)
            else:
                self.image_shape = (3,) + self.target_size
        else:
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (1,)
            else:
                self.image_shape = (1,) + self.target_size
        self.classes = classes
        if class_mode not in {'categorical', 'binary', 'sparse',
                              'input', None}:
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", "input"'
                             ' or None.')
        self.class_mode = class_mode
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        self.interpolation = interpolation
        self.dtype = dtype

        white_list_formats = {'png', 'jpg', 'jpeg', 'bmp',
                              'ppm', 'tif', 'tiff'}
        
        if len(self.img_files)!=len(self.labels):
            raise ValueError('Number of image files and labels should be the same')
            
        self.num_classes = len(self.labels)
        self.samples = len(self.img_files)

        super(ImgListDataGenFrf, self).__init__(self.samples,
                                                batch_size,
                                                shuffle,
                                                seed)



    # Generates data containing batch_size samples
    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros(
            (len(index_array),) + self.image_shape,
            dtype=self.dtype)
        
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.img_files[j]
            img = self.load_img(fname,
                           color_mode=self.color_mode,
                           target_size=self.target_size,
                           interpolation=self.interpolation)
            x = np.array(img)#img_to_array(img, data_format=self.data_format)
            # Pillow images should be closed after `load_img`,
            # but not PIL images.
            if hasattr(img, 'close'):
                img.close()
                
            if self.augmentation:
                aug=augmentation_clss(self.augmentation)
                augmented_img=aug.augment_img(x)
                while(np.array_equal(augmented_img,np.zeros(augmented_img.shape)) == True):  #Avoid black images
                    augmented_img=aug.augment_img(x)
                x = augmented_img
                
              
            if self.rescale:
                x = x.astype(np.float32) * self.rescale
                
            batch_x[i] = x
        # optionally save augmented images to disk for debugging purposes
        if self.save_to_dir:
            for i, j in enumerate(index_array):
                img = batch_x[i]
                fname = '{prefix}_{index}_{hash}.{format}'.format(
                    prefix=self.save_prefix,
                    index=j,
                    hash=np.random.randint(1e7),
                    format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
        # build batch of labels
        if self.class_mode == 'input':
            batch_y = batch_x.copy()
        elif self.class_mode == 'sparse':
            batch_y = self.labels[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.labels[index_array].astype(self.dtype)
        elif self.class_mode == 'categorical':
            batch_y = np.array(keras.utils.to_categorical(y, num_classes=self.n_classes))
        else:
            return batch_x
        return batch_x, batch_y
    
    
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        
        index_array = next(self.index_generator)
        return self._get_batches_of_transformed_samples(index_array)

    
    #load_img code extracted from keras_preprocessing/image.py
    def load_img(self, path, grayscale=False, color_mode='rgb', target_size=None, interpolation='nearest'):
        """Loads an image into PIL format.
        # Arguments
            path: Path to image file.
            color_mode: One of "grayscale", "rbg", "rgba". Default: "rgb".
                The desired image format.
            target_size: Either `None` (default to original size)
                or tuple of ints `(img_height, img_width)`.
            interpolation: Interpolation method used to resample the image if the
                target size is different from that of the loaded image.
                Supported methods are "nearest", "bilinear", and "bicubic".
                If PIL version 1.1.3 or newer is installed, "lanczos" is also
                supported. If PIL version 3.4.0 or newer is installed, "box" and
                "hamming" are also supported. By default, "nearest" is used.
        # Returns
            A PIL Image instance.
        # Raises
            ImportError: if PIL is not available.
            ValueError: if interpolation method is not supported.
        """
        if grayscale is True:
            warnings.warn('grayscale is deprecated. Please use '
                          'color_mode = "grayscale"')
            color_mode = 'grayscale'
        if pil_image is None:
            raise ImportError('Could not import PIL.Image. '
                              'The use of `array_to_img` requires PIL.')
        img = pil_image.open(path)
        if color_mode == 'grayscale':
            if img.mode != 'L':
                img = img.convert('L')
        elif color_mode == 'rgba':
            if img.mode != 'RGBA':
                img = img.convert('RGBA')
        elif color_mode == 'rgb':
            if img.mode != 'RGB':
                img = img.convert('RGB')
        else:
            raise ValueError('color_mode must be "grayscale", "rbg", or "rgba"')
        if target_size is not None:
            width_height_tuple = (target_size[1], target_size[0])
            if img.size != width_height_tuple:
                if interpolation not in _PIL_INTERPOLATION_METHODS:
                    raise ValueError(
                        'Invalid interpolation method {} specified. Supported '
                        'methods are {}'.format(
                            interpolation,
                            ", ".join(_PIL_INTERPOLATION_METHODS.keys())))
                resample = _PIL_INTERPOLATION_METHODS[interpolation]
                img = img.resize(width_height_tuple, resample)
        return img 

In [126]:
import numpy as np
import keras
from PIL import Image as pil_image
from augmentations import augmentation_clss #pip install albumentations
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

def data_generation_(list_imgs_temp, batch_size, target_size = (150,150,3), class_mode = 'binary', 
                    rescale = None, augmentation = None, param_labels = None, idx=index):

    # Initialization
    X = []
    fil=[]
    y = np.empty((batch_size), dtype=int)
    #print(len(y))
    #print(len(list_imgs_temp))
    # Generate data
    for i, img_file in enumerate(list_imgs_temp[idx*batch_size:]):
        # Store sample
        #print(img_file)
        img = np.array(load_img_(path=img_file, target_size=target_size))#skimage.io.imread(img_file
        fil.append(img_file)
        if augmentation:
            aug=augmentation_clss(augmentation)
            augmented_img=aug.augment_img(img)
            while(np.array_equal(augmented_img,np.zeros(augmented_img.shape)) == True):  #Avoid black images
                augmented_img=aug.augment_img(img)
                img = augmented_img
                
              
        if rescale:
            img = img.astype(np.float32) * rescale
                 
        X.append(img)

        # Store label
        if (param_labels is not None):
            y[i] = param_labels[i+idx*batch_size]
        if (i == batch_size-1):
            break
        
    X = np.array(X)
    if class_mode == 'input':
        y = X.copy()
    elif class_mode == 'sparse':
        y = np.array(y)
    elif class_mode == 'binary':
        y = np.array(y).astype(np.float32)
    elif class_mode == 'categorical':
        y = np.array(keras.utils.to_categorical(y, num_classes=n_classes))
            
    return fil, y

In [ ]:
for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    X_train_cv = data[train_idx]
    y_train_cv = labels[train_idx]
    X_valid_cv = data[val_idx]
    y_valid_cv= labels[val_idx]
    
    X_train_cv, y_train_cv = utils.shuffle(X_train_cv, y_train_cv, random_state=1234)
    
    #print(len(X_train_cv)//batch_size)
    for index in range(len(X_train_cv)//batch_size):
        print(index)
        X_train, y_train = data_generation_(X_train_cv, batch_size, target_size = (150,150,3), class_mode = 'binary', 
                    rescale = 1./255, augmentation = None, param_labels = y_train_cv, idx = index)
        print('---Index: {}----'.format(index))
        #print(y_train)
        #print(np.array(X_train))
        print(np.column_stack((np.array(X_train),y_train)))
    


Fold  0
0
---Index: 0----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.189.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.553.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.53.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.102.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.774.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.28.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.266.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.609.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.545.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.205.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.614.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.295.jpg'


---Index: 5----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.795.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.45.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.590.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.774.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.176.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.406.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.174.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.745.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.986.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.362.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.63.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.411.jpg'
  '0.0']
 [

---Index: 11----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.858.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.878.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.536.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.579.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.62.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.523.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.967.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.374.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.922.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.138.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.692.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.273.jpg'
  '0.0']


---Index: 17----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.932.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.756.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.955.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.487.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.246.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.846.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.976.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.97.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.901.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.269.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.41.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.361.jpg'
  '0.0']
 

---Index: 23----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.416.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.232.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.526.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.346.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.994.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.96.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.346.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.595.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.103.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.578.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.393.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.277.jpg'
  '1.0']


...

---Index: 22----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.340.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.582.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.602.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.498.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.660.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.457.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.920.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.903.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.198.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.645.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.59.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.173.jpg'
  '0.0']


---Index: 30----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.785.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.148.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.671.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.169.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.222.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.359.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.718.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.810.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.418.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.86.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.128.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.569.jpg'
  '1.0']


---Index: 38----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.284.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.525.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.694.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.410.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.503.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.13.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.229.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.854.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.442.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.667.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.179.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.197.jpg'
  '1.0']


---Index: 46----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.865.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.36.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.565.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.145.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.842.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.370.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.40.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.861.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.465.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.314.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.326.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.424.jpg'
  '1.0']
 

---Index: 55----
[['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.667.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.524.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.445.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.73.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.472.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.607.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.161.jpg'
  '0.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.789.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.949.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.343.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.694.jpg'
  '1.0']
 ['/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.420.jpg'
  '0.0']


In [ ]:
X_train_cv, y_train_cv = utils.shuffle(X_train_cv, y_train_cv, random_state=12)
    
    #name_weights = "final_model_fold" + str(j) + "_weights.h5"
    #callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)
    training_gen = ImgListDataGen(img_files = X_train_cv, labels=y_train_cv, **train_params)
    validation_gen = ImgListDataGen(img_files = X_valid_cv, labels=y_valid_cv, **val_params) 
    print('Training')                                    
    model = get_model()
    model.fit_generator(
                training_gen,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=10,
                shuffle=True,
                verbose=1,
                validation_data = validation_gen)#,
                #callbacks = callbacks)
    
    print(model.evaluate(X_valid_cv, y_valid_cv))

In [91]:
X_train_cv = data[folds[0][0]]
for i, dat in enumerate(X_train_cv[4:]):
    print(dat)
    print(i)

/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.101.jpg
0
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.102.jpg
1
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.103.jpg
2
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.105.jpg
3
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.107.jpg
4
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.109.jpg
5
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.11.jpg
6
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.110.jpg
7
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.111.jpg
8
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.112.jpg
9
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.113.jpg
10
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.114.jpg
11
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.115.jpg
12
/Users/dfreire/Dropbox/Datasets/small_dataset/train/cats/cat.116.jpg
13
/Us

1217
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.673.jpg
1218
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.674.jpg
1219
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.675.jpg
1220
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.676.jpg
1221
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.677.jpg
1222
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.678.jpg
1223
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.679.jpg
1224
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.680.jpg
1225
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.681.jpg
1226
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.682.jpg
1227
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.683.jpg
1228
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.684.jpg
1229
/Users/dfreire/Dropbox/Datasets/small_dataset/train/dogs/dog.685.jpg
1230
/Users/dfreire/Dropbox/Datasets/s

In [79]:
data

1999